# 체스 데이터를 Parquet 파일로 변환

이 노트북은 `.pgn.zst` 파일을 읽어서 학습 가능한 parquet 파일로 변환합니다.

## 처리 과정
1. `.pgn.zst` 파일을 스트리밍으로 읽기
2. 각 게임에서 샘플 추출
3. 배치 단위로 parquet 파일에 저장
4. 메모리 효율적인 처리

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from preprocessing import extract_samples_from_pgn_zst
import pyarrow as pa
import pyarrow.parquet as pq

## 설정

In [ ]:
# 입력 파일 경로
INPUT_ZST_PATH = "data/lichess_db_standard_rated_2016-12.pgn.zst"

# 출력 디렉토리
OUTPUT_DIR = "data/parquet"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 배치 설정
BATCH_SIZE = 100000  # 한 번에 처리할 샘플 수
MAX_GAMES = None  # None이면 전체 게임 처리
MAX_SAMPLES = None  # None이면 제한 없음

# 필터 설정
MIN_RATING = 1800  # 최소 레이팅 (양쪽 플레이어 모두 이 값 이상)
                   # None이면 필터링 안 함
                   # 예: 2000 = 2000 이상, 1800 = 1800 이상

MIN_TIME_CONTROL = 300  # 최소 시간 제어 (초 단위, 초기 시간)
                        # None이면 필터링 안 함
                        # 예: 300 = 5분 이상, 600 = 10분 이상, 1800 = 30분 이상
                        # 참고: 300초 = 5분, 600초 = 10분, 1800초 = 30분

# Mask 생성 설정
GENERATE_MASK = False  # True면 legal_move_mask 생성, False면 생성 안 함
                       # 학습 시 필요하면 동적으로 생성 가능

# 출력 파일명 패턴
OUTPUT_PATTERN = os.path.join(OUTPUT_DIR, "chess_samples_{:04d}.parquet")

print(f"입력 파일: {INPUT_ZST_PATH}")
print(f"출력 디렉토리: {OUTPUT_DIR}")
print(f"배치 크기: {BATCH_SIZE}")
print(f"\n필터 설정:")
print(f"  최소 레이팅: {MIN_RATING if MIN_RATING else '필터링 안 함'}")
print(f"  최소 시간 제어: {MIN_TIME_CONTROL}초 ({MIN_TIME_CONTROL//60}분)" if MIN_TIME_CONTROL else "  최소 시간 제어: 필터링 안 함")

입력 파일: data/lichess_db_standard_rated_2016-12.pgn.zst
출력 디렉토리: data/parquet
배치 크기: 100000

필터 설정:
  최소 레이팅: 1800
  최소 시간 제어: 300초 (5분)

Mask 생성: 비활성화 (속도 향상)


## 샘플을 DataFrame으로 변환하는 함수

In [ ]:
def samples_to_dataframe(samples):
    """
    샘플 리스트를 pandas DataFrame으로 변환 (NumPy 최적화 버전)
    
    Args:
        samples: [(state, policy, mask, value), ...] 리스트
        
    Returns:
        pandas DataFrame
    """
    n = len(samples)
    
    # NumPy 배열로 미리 할당 (속도 최적화)
    states = np.zeros((n, 18 * 8 * 8), dtype=np.float32)
    policies = np.zeros(n, dtype=np.int32)
    masks = np.zeros((n, 4096), dtype=np.float32)
    values = np.zeros(n, dtype=np.float32)
    
    # 벡터화된 할당
    for i, (state, policy, mask, value) in enumerate(samples):
        states[i] = state.flatten()
        policies[i] = policy
        masks[i] = mask
        values[i] = value
    
    # Parquet 저장을 위해 리스트로 변환 (한 번에 처리)
    return pd.DataFrame({
        'state': [s.tolist() for s in states],
        'policy': policies,
        'mask': [m.tolist() for m in masks],
        'value': values
    })

## 배치 단위로 샘플 추출 및 저장

In [ ]:
def process_and_save_batches(zst_path, output_pattern, batch_size, max_games=None, max_samples=None, 
                             min_rating=None, min_time_control=None):
    """
    .pgn.zst 파일을 배치 단위로 처리하여 parquet 파일로 저장
    
    Args:
        zst_path: 입력 .pgn.zst 파일 경로
        output_pattern: 출력 파일 패턴 (예: "data/parquet/samples_{:04d}.parquet")
        batch_size: 배치 크기
        max_games: 최대 게임 수 (None이면 제한 없음)
        max_samples: 최대 샘플 수 (None이면 제한 없음)
        min_rating: 최소 레이팅 (양쪽 플레이어 모두 이 값 이상)
        min_time_control: 최소 시간 제어 (초 단위, 초기 시간)
    """
    import zstandard as zstd
    import chess.pgn
    import io
    from preprocessing import (
        board_to_tensor,
        legal_move_mask,
        policy_label_from_move,
        game_result_to_value,
        should_include_game
    )
    
    batch_samples = []
    file_idx = 0
    total_samples = 0
    total_games = 0
    skipped_games = 0
    included_games = 0
    
    with open(zst_path, "rb") as f:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(f) as binary_reader:
            # 바이너리 스트림을 텍스트 스트림으로 변환
            text_reader = io.TextIOWrapper(binary_reader, encoding='utf-8', errors='ignore')
            pbar = tqdm(desc="게임 처리 중")
            
            while True:
                game = chess.pgn.read_game(text_reader)
                if game is None:
                    break
                
                # 필터링 조건 확인
                if not should_include_game(game, min_rating, min_time_control):
                    skipped_games += 1
                    total_games += 1
                    pbar.set_postfix({
                        '포함': included_games,
                        '제외': skipped_games,
                        '샘플': total_samples
                    })
                    pbar.update(1)
                    continue
                
                included_games += 1
                board = game.board()
                result = game.headers.get("Result", "*")
                
                # 게임의 각 수마다 샘플 생성
                for move in game.mainline_moves():
                    state = board_to_tensor(board)
                    policy = policy_label_from_move(move)
                    mask = legal_move_mask(board)
                    value = game_result_to_value(result, board.turn)
                    
                    batch_samples.append((state, policy, mask, value))
                    board.push(move)
                    
                    # 배치가 가득 차면 저장
                    if len(batch_samples) >= batch_size:
                        df = samples_to_dataframe(batch_samples)
                        output_path = output_pattern.format(file_idx)
                        # Parquet 저장 최적화 (row_group_size로 I/O 성능 향상)
                        df.to_parquet(
                            output_path, 
                            index=False, 
                            engine='pyarrow',
                            row_group_size=min(50000, len(df))  # 큰 row group으로 I/O 최적화
                        )
                        
                        total_samples += len(batch_samples)
                        file_idx += 1
                        batch_samples = []
                        
                        pbar.set_postfix({
                            '파일': file_idx,
                            '샘플': total_samples,
                            '게임': total_games
                        })
                    
                    # 샘플 수 제한 체크
                    if max_samples and total_samples + len(batch_samples) >= max_samples:
                        break
                
                total_games += 1
                pbar.set_postfix({
                    '포함': included_games,
                    '제외': skipped_games,
                    '샘플': total_samples
                })
                pbar.update(1)
                
                # 게임 수 제한 체크
                if max_games and total_games >= max_games:
                    break
                
                # 샘플 수 제한 체크
                if max_samples and total_samples + len(batch_samples) >= max_samples:
                    break
            
            # 남은 샘플 저장
            if batch_samples:
                df = samples_to_dataframe(batch_samples)
                output_path = output_pattern.format(file_idx)
                df.to_parquet(
                    output_path, 
                    index=False, 
                    engine='pyarrow',
                    row_group_size=min(50000, len(df))
                )
                total_samples += len(batch_samples)
                file_idx += 1
            
            pbar.close()
    
    print(f"\n변환 완료!")
    print(f"  총 게임 수: {total_games:,}")
    print(f"  포함된 게임 수: {included_games:,}")
    print(f"  제외된 게임 수: {skipped_games:,}")
    print(f"  총 샘플 수: {total_samples:,}")
    print(f"  생성된 파일 수: {file_idx + 1}")
    if total_games > 0:
        print(f"  포함 비율: {included_games/total_games*100:.2f}%")
    return total_samples, file_idx + 1

## 변환 실행

In [ ]:
# 파일 존재 확인
if not os.path.exists(INPUT_ZST_PATH):
    print(f"⚠️  입력 파일을 찾을 수 없습니다: {INPUT_ZST_PATH}")
    print("파일 경로를 확인하세요.")
else:
    # 변환 실행
    total_samples, num_files = process_and_save_batches(
        zst_path=INPUT_ZST_PATH,
        output_pattern=OUTPUT_PATTERN,
        batch_size=BATCH_SIZE,
        max_games=MAX_GAMES,
        max_samples=MAX_SAMPLES,
        min_rating=MIN_RATING,
        min_time_control=MIN_TIME_CONTROL
    )

게임 처리 중: 4144it [00:14, 276.17it/s, 포함=263, 제외=3882, 샘플=0]

## 생성된 Parquet 파일 확인

In [ ]:
# 생성된 파일 목록 확인
parquet_files = sorted(Path(OUTPUT_DIR).glob("chess_samples_*.parquet"))
print(f"생성된 파일 수: {len(parquet_files)}")

if parquet_files:
    # 첫 번째 파일 읽어서 확인
    first_file = parquet_files[0]
    print(f"\n첫 번째 파일 확인: {first_file.name}")
    
    df = pd.read_parquet(first_file)
    print(f"  행 수: {len(df)}")
    print(f"  컬럼: {df.columns.tolist()}")
    print(f"\n  첫 번째 샘플:")
    print(f"    Policy: {df.iloc[0]['policy']}")
    print(f"    Value: {df.iloc[0]['value']}")
    print(f"    State shape (평탄화): {len(df.iloc[0]['state'])} (예상: 1152)")
    print(f"    Mask shape (평탄화): {len(df.iloc[0]['mask'])} (예상: 4096)")
    
    # State 복원 테스트
    state_flat = np.array(df.iloc[0]['state'], dtype=np.float32)
    state_restored = state_flat.reshape(18, 8, 8)
    print(f"    State 복원 shape: {state_restored.shape}")
    
    # Mask 복원 테스트
    mask_flat = np.array(df.iloc[0]['mask'], dtype=np.float32)
    mask_restored = mask_flat.reshape(4096)
    print(f"    Mask 복원 shape: {mask_restored.shape}")
    print(f"    합법 수 개수: {int(mask_restored.sum())}")

## Parquet 파일에서 데이터 로드하는 함수 (학습용)

In [ ]:
def load_sample_from_parquet(df_row):
    """
    Parquet 파일의 한 행을 원래 형태로 복원
    
    Args:
        df_row: pandas DataFrame의 한 행
        
    Returns:
        (state, policy, mask, value) 튜플
    """
    state = np.array(df_row['state'], dtype=np.float32).reshape(18, 8, 8)
    policy = int(df_row['policy'])
    mask = np.array(df_row['mask'], dtype=np.float32)
    value = float(df_row['value'])
    
    return state, policy, mask, value


# 사용 예시
if parquet_files:
    print("샘플 로드 테스트:")
    df = pd.read_parquet(parquet_files[0])
    state, policy, mask, value = load_sample_from_parquet(df.iloc[0])
    
    print(f"  State shape: {state.shape}")
    print(f"  Policy: {policy}")
    print(f"  Mask shape: {mask.shape}, 합법 수: {int(mask.sum())}")
    print(f"  Value: {value}")

## PyTorch Dataset 클래스 (Parquet 파일용)

In [ ]:
import torch
from torch.utils.data import Dataset
from pathlib import Path


class ParquetChessDataset(Dataset):
    """
    Parquet 파일에서 체스 데이터를 로드하는 Dataset
    
    여러 parquet 파일을 하나의 Dataset으로 처리합니다.
    """
    
    def __init__(self, parquet_dir, file_pattern="chess_samples_*.parquet"):
        """
        Args:
            parquet_dir: parquet 파일이 있는 디렉토리
            file_pattern: 파일 패턴
        """
        self.parquet_dir = Path(parquet_dir)
        self.parquet_files = sorted(self.parquet_dir.glob(file_pattern))
        
        if not self.parquet_files:
            raise ValueError(f"Parquet 파일을 찾을 수 없습니다: {parquet_dir}/{file_pattern}")
        
        # 각 파일의 행 수를 미리 계산
        self.file_lengths = []
        self.cumulative_lengths = [0]
        
        print(f"Parquet 파일 로드 중... ({len(self.parquet_files)}개 파일)")
        for file_path in tqdm(self.parquet_files, desc="파일 인덱싱"):
            df = pd.read_parquet(file_path)
            length = len(df)
            self.file_lengths.append(length)
            self.cumulative_lengths.append(self.cumulative_lengths[-1] + length)
        
        self.total_length = self.cumulative_lengths[-1]
        print(f"총 샘플 수: {self.total_length:,}")
    
    def __len__(self):
        return self.total_length
    
    def __getitem__(self, idx):
        # 어떤 파일에 속하는지 찾기
        file_idx = 0
        for i, cum_len in enumerate(self.cumulative_lengths[1:], 1):
            if idx < cum_len:
                file_idx = i - 1
                break
        
        # 파일 내 상대 인덱스
        local_idx = idx - self.cumulative_lengths[file_idx]
        
        # 파일 로드 (캐싱 가능하지만 메모리 고려)
        df = pd.read_parquet(self.parquet_files[file_idx])
        row = df.iloc[local_idx]
        
        # 데이터 복원
        state, policy, mask, value = load_sample_from_parquet(row)
        
        return (
            torch.from_numpy(state).float(),
            torch.tensor(policy, dtype=torch.long),
            torch.from_numpy(mask).float(),
            torch.tensor(value, dtype=torch.float32),
        )


# Dataset 테스트
if parquet_files:
    print("\nDataset 테스트:")
    dataset = ParquetChessDataset(OUTPUT_DIR)
    print(f"  Dataset 크기: {len(dataset):,}")
    
    # 첫 번째 샘플 확인
    state, policy, mask, value = dataset[0]
    print(f"  첫 번째 샘플:")
    print(f"    State: {state.shape}, dtype: {state.dtype}")
    print(f"    Policy: {policy}, dtype: {policy.dtype}")
    print(f"    Mask: {mask.shape}, dtype: {mask.dtype}")
    print(f"    Value: {value}, dtype: {value.dtype}")

## DataLoader 사용 예시

In [ ]:
from torch.utils.data import DataLoader

if parquet_files:
    # Dataset 생성
    dataset = ParquetChessDataset(OUTPUT_DIR)
    
    # DataLoader 생성
    dataloader = DataLoader(
        dataset,
        batch_size=32,
        shuffle=True,
        num_workers=0  # Windows에서는 0 권장
    )
    
    # 배치 확인
    print("DataLoader 테스트:")
    for batch_idx, (states, policies, masks, values) in enumerate(dataloader):
        print(f"  배치 {batch_idx + 1}:")
        print(f"    States: {states.shape}")
        print(f"    Policies: {policies.shape}")
        print(f"    Masks: {masks.shape}")
        print(f"    Values: {values.shape}")
        
        if batch_idx >= 2:  # 처음 3개 배치만 출력
            break
    
    print("\n✅ Parquet 파일이 학습 가능한 형태로 준비되었습니다!")